In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from Element_PI import VariancePersistv1
import glob
from sklearn.model_selection import train_test_split


Rips(maxdim=1, thresh=inf, coeff=2, do_cocycles=False, n_perm = None, verbose=True)


In [2]:
px = 32
py = 32
spread = 1      #varience or std, not sure
axis_bound = 2.5  #angstroms
samples = 500

data_path = '/Users/knp/projects/iMoDELS-supplements/data/raw-data/everything.csv'
data = pd.read_csv(data_path, index_col=0)
data = data.drop(['backbone'],axis=1)

molecules = glob.glob('/Users/knp/projects/terminal_group_screening/src/util/molecules/*.xyz')
molecules = list(set(molecules))
im_mols = {}
for m in molecules:
    mol_name = m.split('/')[-1].split('.')[0]
    if 'ch3' in mol_name:
        mol_name = mol_name.split('-')[0]
        im_mols[mol_name] = VariancePersistv1(m, pixelx=px, pixely=py, myspread=spread, myspecs={"maxBD": axis_bound, "minBD":-.10}, showplot=False).reshape(1, px,py, 1)
        im_mols[mol_name] = np.asarray(im_mols[mol_name]).astype('float32')
def str_to_im(chem_name):
    if chem_name == 'difluoromethyl' or chem_name == 'phenol' or chem_name == 'toluene':
        return
    else:
        return im_mols[chem_name]
data.terminal_group_1 = data.terminal_group_1.apply(str_to_im)
data.terminal_group_2 = data.terminal_group_2.apply(str_to_im)
data.terminal_group_3 = data.terminal_group_3.apply(str_to_im)
data = data.dropna()


msk = np.random.rand(len(data)) < 0.8
train, test = data[msk] , data[~msk]

train_X, train_y = train[['terminal_group_1', 'terminal_group_2', 'terminal_group_3']], train['COF']
test_X, test_y = test[['terminal_group_1', 'terminal_group_2', 'terminal_group_3']], test['COF']

In [5]:
input_t1 = keras.Input(shape=(px, py, 1))
x = keras.layers.Conv2D(16, 5)(input_t1)
x = keras.layers.Conv2D(16, 4)(x)
x = keras.layers.Conv2D(16, 3)(x)
x = keras.layers.Dense(16)(x)
out = keras.layers.Dense(1,activation='linear')(x)
model = keras.Model(inputs = input_t1, outputs=out)
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])
model.fit(np.array(train_X['terminal_group_1'].to_list()), np.array(train_y.to_list()), epochs=5, verbose=1)

In [4]:
np.array(train_X['terminal_group_1'].to_list()).shape

(4647, 1, 32, 32, 1)

In [135]:
input_t1 = keras.Input(shape=(px, py, 1))
x = keras.layers.Conv2D(16, 5)(input_t1)
x = keras.layers.Conv2D(16, 4)(x)
x = keras.layers.Conv2D(16, 3)(x)
model_x = keras.Model(inputs = input_t1, outputs=x)

input_t2 = keras.Input(shape=(px, py, 1))
y = keras.layers.Conv2D(16, 5)(input_t2)
y = keras.layers.Conv2D(16, 4)(y)
y = keras.layers.Conv2D(16, 3)(y)
model_y = keras.Model(inputs = input_t2, outputs=y)

input_t3 = keras.Input(shape=(px, py, 1))
z = keras.layers.Conv2D(16, 5)(input_t3)
z = keras.layers.Conv2D(16, 4)(z)
z = keras.layers.Conv2D(16, 3)(z)
model_z = keras.Model(inputs = input_t3, outputs=z)

combo = keras.layers.concatenate([model_x.output, model_y.output, model_z.output])

out = keras.layers.Dense(16, activation='relu')(combo)
out = keras.layers.Dense(1, activation='linear')(out)

model = keras.Model(inputs=[input_t1, input_t2, input_t3], outputs=out)
model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

In [138]:
model.fit(x = [np.array(train_X['terminal_group_1']), np.array(train_X['terminal_group_2']), np.array(train_X['terminal_group_3'])], y = np.array(train_y))

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [129]:
mdoel

(None, 32, 32, 1) <dtype: 'float32'>
(None, 32, 32, 1) <dtype: 'float32'>
(None, 1, 32, 32) <dtype: 'float32'>


[None, None, None]